In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import math
import numpy as np
import pandas as pd
import os

In [3]:
## 读取睡眠标签及观察值序列文件hmm_obsv.txt 
cwd = os.getcwd()
HMM_data_path = os.path.join(cwd, 'HMM_data')

file =  os.path.join(HMM_data_path, 'hmm_obsv.txt') 
fid = open(file,"r")
note = fid.readlines()
fid.close()

states = [] # 存储hmm_obsv.txt的睡眠标签

T = 960 #设置观察序列长度T=2*60*8=960
observations_data = np.empty((len(note),T)) #用来存储观察值序列的矩阵，每行为一个观察值序列，每个序列长度固定为T=960
Ti = 0

for line in note:
    line_rr = line.split(",")
    states.append(line_rr[0])#将睡眠标签写进states数组
    line_new = [] # 对应RR间期
    for content in line_rr[1:-1]:  #去掉第一位标签，最后一位换行符
        line_new.append(int(content)) #int 的作用是将str转为int

#     print(line_rr[0])
#     print(line_new)
#     print("==================")

    # 为了保证观察序列长度一致，设置观察序列长度T=2*60*8=960，因此从序列最后开始倒数选取T个观察值
    #（why倒数选取T个观察值：因为越后面的RR间期越能反应当前的睡眠分期）
    line_new = line_new[-T:]
    observations_data[Ti:] = line_new
    Ti+=1

print(observations_data)


[[ 2.  1.  0. ...  7.  6.  5.]
 [-6. -8. -9. ... 14. 14. 15.]
 [ 5.  3.  3. ...  5.  6.  8.]
 ...
 [-2. -4. -3. ...  3.  2.  2.]
 [ 0. -2. -3. ...  8.  8.  8.]
 [10.  9.  9. ...  9.  6.  3.]]


In [5]:
observations = list(range(-37,38))
Dic_observations = {observations[i]:i for i in range(75)}


In [8]:
# 利用Baum-Welch算法求解HMM模型参数(HMM问题二)
# 由于Baum-Welch(鲍姆-韦尔奇)算法是基于EM算法的近似算法，所以我们需要多跑几次，比如下面我们跑三次，选择一个比较优的模型参数，代码如下：
import numpy as np
from hmmlearn import hmm
import warnings
warnings.filterwarnings("ignore")

# states = ["box 1", "box 2", "box3"]
states = ['1','2','3','4','R','W']
n_states = len(states)

# observations = ["red", "white"]
observations = list(range(-37,38))
Dic_observations = {observations[i]:i for i in range(75)}
observations_data_index = np.empty(np.shape(observations_data))
for line in range(225):
    for colum in range(960):
        observations_data_index[line][colum]=Dic_observations[observations_data[line][colum]]

n_observations = len(observations)
# hmmlearn实现了三种HMM模型类，按照观测状态是连续状态还是离散状态，可以分为两类。
# GaussianHMM和GMMHMM是连续观测状态的HMM模型，而MultinomialHMM是离散观测状态的模型，也是我们在HMM原理系列篇里面使用的模型。

# model = hmm.MultinomialHMM(n_components=n_states, n_iter=20, tol=0.01) #不满足多项式分布
model = hmm.GaussianHMM(n_components=n_states, covariance_type='diag', n_iter=1000, tol=0.01) #本模型中observations为连续状态，满足高斯分布
X = observations_data_index

print("\n正在训练隐马尔科夫模型....")  
model.fit(X)
print(model.startprob_)
print(model.transmat_)
# print(model.emissionprob_)
print(model.score(X))
print("========")
model.fit(X)
print(model.startprob_)
print(model.transmat_)
# print(model.emissionprob_)
print(model.score(X))
print("========")
model.fit(X)
model.fit(X)
print(model.startprob_)
print(model.transmat_)
# print(model.emissionprob_)
print(model.score(X))
print("========")
# 最终我们会选择分数最高的模型参数。



正在训练隐马尔科夫模型....
[6.40453845e-267 0.00000000e+000 1.00000000e+000 0.00000000e+000
 0.00000000e+000 1.69342240e-271]
[[1.75492386e-001 1.57923179e-001 1.40174151e-001 1.93017133e-001
  1.75470114e-001 1.57923037e-001]
 [1.84183268e-001 3.68433307e-001 1.57899994e-001 2.84922399e-144
  2.63166657e-002 2.63166766e-001]
 [1.62164521e-001 1.08109688e-001 1.89177342e-001 5.40548439e-002
  3.51356496e-001 1.35137110e-001]
 [2.35294118e-001 6.12603393e-024 1.17647059e-001 1.64130139e-234
  5.29411765e-001 1.17647059e-001]
 [5.47218245e-001 7.13778739e-002 7.16820271e-002 4.76053900e-002
  1.43102980e-001 1.19013484e-001]
 [2.42424266e-001 2.42424279e-001 3.03030333e-001 6.06060666e-002
  9.09090998e-002 6.06059552e-002]]
-640595.5250371859
[0.00000000e+000 6.93091083e-254 0.00000000e+000 1.00000000e+000
 1.02137748e-307 0.00000000e+000]
[[5.00000000e-001 1.03126329e-039 0.00000000e+000 3.80482294e-247
  5.00000000e-001 0.00000000e+000]
 [1.21951327e-002 6.21951506e-001 6.09756631e-002 1.219507

In [9]:
# Predict
print(model.score([observations_data_index[0]]))

-3040.7282683274534


[1, 2, 3]